In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.interpretable_functions as interpret
import utils.RiskSLIM as slim
import utils.stumps as stumps
import utils.fairness_functions as fairness
import utils.risktool as risktool
from utils.load_settings import load_settings

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model

# restore saved variables
%store -r summary_property2_KY_interpret

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### CART & EBM

In [2]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv").sort_values('person_id')
x = data.loc[:,:'current_violence20']
y = data['property_two_year'].values

In [3]:
#### CART
depth = [5,6,7,8,9,10]
cart_summary = interpret.CART(X=x,
                         Y=y,
                         depth=depth,
                         seed=816)

#### EBM
estimators = [40]
depth = [2]
learning_rate = [0.7]
ebm_summary = interpret.EBM(X=x,
                       Y=y,
                       learning_rate=learning_rate,
                       depth=depth,
                       estimators=estimators,
                       seed =816)

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia 

In [4]:
np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])

(0.775524603006789, 0.013732743684478054)

In [5]:
np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])

(0.7835999549167814, 0.0054843039729263856)

## Lasso Stumps

In [6]:
## load stumps & original data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv").sort_values('person_id')
original_data = original_data.loc[:, ['person_id', 'age_at_current_charge', 'p_charges']]
data = pd.merge(original_data, data, on='person_id')

## split X & Y
X_stumps = data.loc[:,:'current_violence201'].copy()
Y_stumps = data['property_two_year'].values.copy()
Y_stumps[Y_stumps == -1] = 0
cols = X_stumps.columns[5:]

## load train & test stumps data
train_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_train_stumps.csv")
test_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_test_stumps.csv")
X_train_stumps = train_stumps.loc[:,:'current_violence201'].copy()
X_test_stumps = test_stumps.loc[:,:'current_violence201'].copy()
Y_train_stumps = train_stumps['property_two_year'].values.copy()
Y_test_stumps = test_stumps['property_two_year'].values.copy()
Y_train_stumps[Y_train_stumps == -1] = 0
Y_test_stumps[Y_test_stumps == -1] = 0

##### Single Stump Model

In [7]:
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.002, 
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

23

In [8]:
single_stump_model['features']

['sex1',
 'age_at_current_charge20',
 'age_at_current_charge21',
 'age_at_current_charge24',
 'age_at_current_charge27',
 'age_at_current_charge35',
 'age_at_current_charge39',
 'age_at_current_charge43',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest6',
 'p_arrest7',
 'p_felony1',
 'p_felony2',
 'p_misdemeanor1',
 'p_property1',
 'p_drug1',
 'p_stealing1',
 'ADE1',
 'p_pending_charge1',
 'p_probation1',
 'p_incarceration1']

#### Nested Cross Validation

In [9]:
stump_summary = stumps.stump_cv(X = X_stumps, 
                                Y = Y_stumps, 
                                columns=cols, 
                                c_grid={'C': [0.001, 0.002]}, 
                                seed = 816)

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [10]:
stump_summary['best_params'], np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])

([{'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}],
 0.7817697488560319,
 0.0007774753577442616)

### RiskSLIM

In [11]:
## load stumps data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_stumps.csv").sort_values('person_id')

## single stump model
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.001,  
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

18

In [12]:
### Subset features
selected_features = single_stump_model['features']
if 'sex1' in selected_features:
    selected_features = ['property_two_year', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['property_two_year', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0

sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on='person_id')
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [13]:
riskslim_summary = slim.risk_nested_cv(X=sub_X, 
                                       Y=sub_Y, 
                                       indicator = indicator,
                                       y_label='property_two_year', 
                                       max_coef=5, 
                                       max_coef_number=5, 
                                       max_runtime=1000,
                                       max_offset=100,
                                       c=1e-6, 
                                       seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
12/20/19 @ 10:19 AM | 233 rows in lookup table
12/20/19 @ 10:19 AM | ------------------------------------------------------------
12/20/19 @ 10:19 AM | runnning initialization procedure
12/20/19 @ 10:19 AM | ------------------------------------------------------------
12/20/19 @ 10:19 AM | CPA produced 2 cuts
12/20/19 @ 10:19 AM | running naive rounding on 6 solutions
12/20/19 @ 10:19 AM | best objective value: 0.5684
12/20/19 @ 10:19 AM | rounding produced 5 integer solutions
12/20/19 @ 10:19 AM | best objective value is 0.5305
12/20/19 @ 10:19 AM | running sequential rounding on 6 solutions
12/20/19 @ 10:19 AM | best objective value: 0.5684
12/20/19 @ 10:19 AM | sequential rounding produced 3 integer solutions
12/20/19 @ 10:19 AM | best objective value: 0.3618
12/20/19 @ 10:19 AM | polishing 8 solutions
12/20/19 @ 10:19 AM | best objective value: 0.3618
12/20/19 @ 10:19 AM | polishing produced 4 integer solutions
12/20/19 @ 1

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/20/19 @ 10:19 AM | adding 293 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2136.
Tried aggregator 1 time.
Reduced MIP has 38 rows, 40 columns, and 111 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

  81275 20413        0.2083    14        0.2098        0.2047   756871    2.43%           rho_3 D  81275  44650     22
  82933 20541        0.2062    27        0.2098        0.2048   768425    2.39%           rho_4 D  82933  44375     13
  84556 20589        0.2082    11        0.2098        0.2049   780069    2.34%          rho_15 U  84556  84555     23
  86130 20642        0.2058    14        0.2098        0.2050   791765    2.29%           rho_0 D  86130  12503     26
  87719 20695        0.2070     8        0.2098        0.2051   803553    2.24%          rho_13 D  87719  87718     26
  89282 20719        0.2072    13        0.2098        0.2052   814867    2.20%          rho_16 D  89282  58491     23
  90858 20685        0.2069    13        0.2098        0.2053   826207    2.14%           rho_4 D  90858  90857     21
  92423 20696        0.2076    24        0.2098        0.2054   837373    2.10%          rho_11 D  92423  57727     19
  94000 20692        0.2065    10        0.2098 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/20/19 @ 10:21 AM | adding 328 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2132.
Tried aggregator 1 time.
Reduced MIP has 38 rows, 40 columns, and 111 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

  82665 24744        0.2086     8        0.2096        0.2032   718603    3.07%           rho_8 D  82665  82663     34
  84200 25010        0.2063    27        0.2096        0.2033   728904    3.03%           rho_4 D  84200  41400     12
  85764 25348        0.2082    13        0.2096        0.2034   738855    2.98%           rho_4 N  85764  47490     26
Elapsed time = 41.78 sec. (41273.64 ticks, tree = 7.91 MB, solutions = 10)
  87246 25596        0.2059    14        0.2096        0.2035   749100    2.94%          rho_14 U  87246  87245     31
  88710 25806        0.2043    15        0.2096        0.2035   759015    2.91%           rho_2 D  88710  88709     32
  90198 25971        0.2094    13        0.2096        0.2036   768621    2.87%          rho_14 D  90198  90197     25
  91692 26210        0.2067     7        0.2096        0.2037   778219    2.83%           rho_8 D  91692  91690     34
  93117 26350        0.2069    22        0.2096        0.2038   788291    2.79%           rh

...

  88689 27838        cutoff              0.2101        0.2032   791953    3.28%         alpha_6 U  88689  34487     27
  90319 28145        0.2099     1        0.2101        0.2033   803566    3.23%        alpha_14 D  90319  90318     48
  91939 28373        0.2091    12        0.2101        0.2034   815177    3.19%           rho_5 N  91939  51564     28
  93602 28615        cutoff              0.2101        0.2034   826450    3.15%          rho_10 N  93602  55908     24
  95181 28725        cutoff              0.2101        0.2035   837794    3.12%          rho_12 N  95181  95180     27
  96791 28946        0.2052    16        0.2101        0.2036   848937    3.08%          rho_11 N  96791  96790     23
  98371 29115        0.2062    16        0.2101        0.2037   860315    3.04%           rho_5 D  98371  98370     33
  99920 29225        0.2072    13        0.2101        0.2038   871185    3.01%          rho_12 U  99920  99919     32
 101474 29344        cutoff              0.2101 

12/20/19 @ 11:11 AM | polishing produced 4 integer solutions
12/20/19 @ 11:11 AM | initialization produced 12 feasible solutions
12/20/19 @ 11:11 AM | best objective value: 0.2135
12/20/19 @ 11:11 AM | ------------------------------------------------------------
12/20/19 @ 11:11 AM | completed initialization procedure
12/20/19 @ 11:11 AM | ------------------------------------------------------------
12/20/19 @ 11:11 AM | 233 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.20223837922045668
CPXPARAM_MIP_Tolerances_UpperCutoff              0.21347777589487704


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/20/19 @ 11:11 AM | adding 288 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2135.
Tried aggregator 1 time.
Reduced MIP has 38 rows, 40 columns, and 111 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Elapsed time = 35.72 sec. (41266.94 ticks, tree = 7.67 MB, solutions = 7)
  90552 25217        0.2073    15        0.2098        0.2030   748494    3.25%           rho_1 U  90552  90550     13
  91832 25313        0.2091    20        0.2098        0.2031   757885    3.22%           rho_3 N  91832  39836     24
  93070 25399        0.2044    16        0.2098        0.2031   767029    3.18%          rho_12 D  93070  93068     26
  94366 25509        cutoff              0.2098        0.2032   776122    3.14%          rho_11 U  94366  51846     13
  95584 25580        0.2046    16        0.2098        0.2033   785373    3.10%           rho_5 D  95584  95583     27
  96886 25685        cutoff              0.2098        0.2034   794273    3.07%          rho_10 U  96886  70879     35
  98140 25756        0.2065    17        0.2098        0.2035   803525    3.03%           rho_3 D  98140  83404     25
  99382 25847        cutoff              0.2098        0.2036   812473    2.99%           rho

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [14]:
np.mean(riskslim_summary['validation_auc']), np.mean(riskslim_summary['test_auc'])

(0.7465183899061798, 0.7465890942326544)

#### Single RiskSLIM Model

In [15]:
selected_features = ["property_two_year"] + single_stump_model['features']
sub_train_data = train_stumps[selected_features]
sub_test_data = test_stumps[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
sub_cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y+
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [16]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': sub_cols,
    'outcome_name': 'property_two_year',
    'sample_weights': sample_weights
}

In [17]:
model_info, mip_info, lcpa_info = slim.risk_slim(new_train_data, 
                                                 max_coefficient=5, 
                                                 max_L0_value=5, 
                                                 c0_value=1e-6, 
                                                 max_offset=100, 
                                                 max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
12/20/19 @ 11:13 AM | 233 rows in lookup table
12/20/19 @ 11:13 AM | ------------------------------------------------------------
12/20/19 @ 11:13 AM | runnning initialization procedure
12/20/19 @ 11:13 AM | ------------------------------------------------------------
12/20/19 @ 11:13 AM | CPA produced 2 cuts
12/20/19 @ 11:13 AM | running naive rounding on 6 solutions
12/20/19 @ 11:13 AM | best objective value: 0.5648
12/20/19 @ 11:13 AM | rounding produced 5 integer solutions
12/20/19 @ 11:13 AM | best objective value is 0.5723
12/20/19 @ 11:13 AM | running sequential rounding on 6 solutions
12/20/19 @ 11:13 AM | best objective value: 0.5648
12/20/19 @ 11:13 AM | sequential rounding produced 3 integer solutions
12/20/19 @ 11:13 AM | best objective value: 0.3629
12/20/19 @ 11:13 AM | polishing 8 solutions
12/20/19 @ 11:13 AM | best objective value: 0.3629
12/20/19 @ 11:13 AM | polishing produced 5 integer solutions
12/20/19 @ 1

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/20/19 @ 11:13 AM | adding 304 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2134.
Tried aggregator 1 time.
Reduced MIP has 38 rows, 40 columns, and 111 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.02 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Elapsed time = 27.99 sec. (41264.15 ticks, tree = 11.04 MB, solutions = 7)
 113050 35246        0.2051    14        0.2099        0.2035   875919    3.07%          rho_12 U 113050  69680     28
 115063 35632        0.2071    12        0.2099        0.2035   888686    3.04%           rho_9 D 115063 115061     27
 117006 35807        0.2072    13        0.2099        0.2036   901270    3.00%          rho_12 D 117006  90974     30
 118939 36005        0.2085    15        0.2099        0.2037   914103    2.96%           rho_4 U 118939 118937     28
 120890 36215        cutoff              0.2099        0.2038   926639    2.92%        alpha_16 U 120890 120888     23
 122892 36448        cutoff              0.2099        0.2038   938873    2.89%           rho_7 U 122892 122890     31
 124838 36604        cutoff              0.2099        0.2039   951016    2.85%           rho_0 D 124838 124837     36
 126756 36773        0.2063    16        0.2099        0.2040   963252    2.82%          rho

In [18]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [19]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.7464116321271687

In [20]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.7569549443741534

### Arnold PSA

In [22]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv").sort_values('person_id')
settings = load_settings()
for decoder_name, decoder_dict in settings['decoders'].items():
    data = data.replace({decoder_name: decoder_dict})  
    
# compute scaled arnold scores
for score in ['arnold_nca' , 'arnold_nvca', 'arnold_fta']:
    data[score] = data[score + "_raw"]
    data = data.replace({score: settings[score + '_scaler'] })
    if score != 'arnold_nvca':
        data[score] = data[score].astype(int)
X_arnold = data.loc[:,['arnold_nca', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['property_two_year'].values

In [23]:
arnold = risktool.risktool(X_arnold, Y_arnold, label='arnold_nca')

In [24]:
np.mean(arnold['auc'])

0.7104868868068501

#### Single Arnold PSA

In [25]:
### load data
test_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_test.csv").sort_values('person_id')
settings = load_settings()
for decoder_name, decoder_dict in settings['decoders'].items():
    test_data = test_data.replace({decoder_name: decoder_dict})  
    
# compute scaled arnold scores
for score in ['arnold_nca' , 'arnold_nvca', 'arnold_fta']:
    test_data[score] = test_data[score + "_raw"]
    test_data = test_data.replace({score: settings[score + '_scaler'] })
    if score != 'arnold_nvca':
        test_data[score] = test_data[score].astype(int)

In [26]:
X = test_data['arnold_nca'].values
Y = test_data['property_two_year'].values
roc_auc_score(Y, X)

0.7245532029420475

### Results

In [27]:
#### save results
summary_property2_KY_interpret = {"cart": cart_summary,
                                  "ebm": ebm_summary, 
                                  'stumps': stump_summary, 
                                  'riskslim': riskslim_summary, 
                                  'arnold': arnold}
%store summary_property2_KY_interpret

Stored 'summary_property2_KY_interpret' (dict)


In [28]:
results = [["cart", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])],
           ["ebm", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])], 
           ["stumps", np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])],
           ['riskslim', np.mean(riskslim_summary['test_auc'])],
           ['arnold', np.mean(arnold['auc'])]]
results

[['cart', 0.775524603006789, 0.013732743684478054],
 ['ebm', 0.7835999549167814, 0.0054843039729263856],
 ['stumps', 0.7817697488560319, 0.0007774753577442616],
 ['riskslim', 0.7465890942326544],
 ['arnold', 0.7104868868068501]]

In [29]:
auc = [np.mean(cart_summary['holdout_test_auc']), 
       np.mean(ebm_summary['holdout_test_auc']), 
       np.mean(stump_summary['holdout_test_auc']), 
       np.mean(riskslim_summary['test_auc'])]

In [30]:
path = "./kentucky/logs/interpretable/"
results = [["Property", np.str((round(np.mean(cart_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(ebm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(ebm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(stump_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(stump_summary['holdout_test_auc']), 3)) + ")",             
            np.str(round(np.mean(riskslim_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_summary['test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3),
            np.str(round(np.mean(arnold['auc']), 3)) + " (" + np.str(round(np.std(arnold['auc']),3)) + ")"]]
with open(path + 'KY-two-year-interpretable-summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

### Save Fairness Matrix

In [31]:
matrix = ['confusion_matrix_stats', 'calibration_stats', 'race_auc', 'condition_pn', 'no_condition_pn']
name = ['confusion', 'calibration', 'race-auc', 'condition-pn', 'no-condition-pn']

In [32]:
for i in range(len(matrix)):
    
    if ( (i == 0) | (i == 1) ):
        
        path = './kentucky/fairness/' + name[i] + '/two-year/property/'
        ## confusion matrix and calibration only applies to cart, ebm, riskslim and stumps
        cart_matrix = cart_summary[matrix[i]]
        ebm_matrix = ebm_summary[matrix[i]]
        riskslim_matrix = riskslim_summary[matrix[i]]
        stumps_matrix = stump_summary[matrix[i]]
        
        ## save results
        cart_matrix.to_csv(path+'cart-'+name[i]+'.csv', index=None,header=True)
        ebm_matrix.to_csv(path+'ebm-'+name[i]+'.csv', index=None,header=True)
        riskslim_matrix.to_csv(path+'riskslim-'+name[i]+'.csv', index=None,header=True)
        stumps_matrix.to_csv(path+'stumps-'+name[i]+'.csv', index=None,header=True)
        
    else:
        path = './kentucky/fairness/' + name[i] + '/two-year/property/'
        
        ## including arnold and compas now
        cart_matrix = cart_summary[matrix[i]]
        ebm_matrix = ebm_summary[matrix[i]]
        riskslim_matrix = riskslim_summary[matrix[i]]
        stumps_matrix = stump_summary[matrix[i]]
        arnold_matrix = arnold[matrix[i]]
        
        ## save results
        cart_matrix.to_csv(path+'cart-'+name[i]+'.csv', index=None,header=True)
        ebm_matrix.to_csv(path+'ebm-'+name[i]+'.csv', index=None,header=True)
        riskslim_matrix.to_csv(path+'riskslim-'+name[i]+'.csv', index=None,header=True)
        stumps_matrix.to_csv(path+'stumps-'+name[i]+'.csv', index=None,header=True)
        arnold_matrix.to_csv(path+'arnold-'+name[i]+'.csv', index=None,header=True)